In [105]:
import numpy as np
import pandas as pd
import csv
from math import factorial
from sklearn.metrics import roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
import time
from sklearn.utils import shuffle

In [106]:
def pos(i,prod,L):
    '''
    Compute positive function and gradient information
    
    input:
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fpt - positive function value
        gfpt - positive function gradient
    '''
    fpt = 0.0 
    gfpt = 0.0 
    fpt = (L/2+prod)**i 
    gfpt = i*(L/2+prod)**(i-1) # no xt yet!
    hfpt = i*(i-1)*(L/2+prod)**(i-2)
    return fpt,gfpt,hfpt               

In [107]:
def comb(n, k):
    '''
    Compute combination
    
    input:
        n - total number
        k - number of chosen
    
    output:
        c - number of combination
    '''
    return factorial(n) / factorial(k) / factorial(n - k)

In [108]:
def neg(loss,i,prod,L):
    '''
    Compute negative function and gradient information
    
    input:
        loss - loss function
        i - index of function
        t - iteration
        prod - wt*xt
        
    output:
        fnt - negative function value
        gfnt - negative function gradient
    '''
    fnt = 0.0 # n stands for negative
    gfnt = 0.0
    hfnt = 0.0
    for k in range(i,N+1):
        # compute forward difference
        delta = 0.0
        for j in range(k+1):
            delta += (-1)**(k-j)*comb(k,j)*loss(j/N)
        # compute coefficient
        beta = comb(N,k)*comb(k,i)*(N+1)*delta/(2*L)**k
        # compute function value
        fnt += beta*(L/2-prod)**(k-i)
        # compute gradient
        gfnt += beta*(k-i)*(L/2-prod)**(k-i-1)  # no xt yet!
        # compute hessian
        hfnt += beta*(k-i)*(k-i-1)*(L/2-prod)**(k-i-2)
    return fnt,gfnt,hfnt

In [109]:
def w_grad(gfpt,gfnt,yt,at,bt,alphat):
    '''
    Gradient with respect to w
    
    input:
        fpt - positive function at t
        gfpt - positive function gradient at t
        fnt - negative function at t
        gfnt - negative function gradient at t
        yt - sample label at t
        pt - p at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        gradwt - gradient w.r.t. w at t
    '''
    gradwt = 0.0
    if yt == 1:
        gradwt = 2*(alphat - at)*gfpt
    else:
        gradwt = 2*(alphat - bt)*gfnt
    return gradwt

In [110]:
def w_hess(hfpt,hfnt,yt,at,bt,alphat):
    hesswt = 0.0
    if yt == 1:
        hesswt = 2*(alphat - at)*hfpt
    else:
        hesswt = 2*(alphat - bt)*hfnt
    return hesswt

In [111]:
def proj(wt,R):
    '''
    Projection
    
    input:
        wt - w at t
        R - radius
        
    output:
        proj - projected wt
    '''
    norm = np.linalg.norm(wt)
    if norm > R:
        wt = wt/norm*R
    return wt

In [112]:
def a_grad(fpt,yt,at):
    '''
    Gradient with respect to a
    
    input:
        fpt - positive function at t
        yt - sample label at t
        pt - p at t
        at - a at t
    
    output:
        gradat - gradient w.r.t a at t
    '''
    gradat = 0.0 
    if yt == 1:
        gradat = 2*(at - fpt)
    else:
        gradat = 2*at
    return gradat

In [113]:
def b_grad(fnt,yt,bt):
    '''
    Gradient with respect to b
    
    input:
        fnt - negative function at t
        yt - sample label at t
        pt - p at t
        bt - b at t
    
    output:
        gradbt - gradient w.r.t b at t
    '''
    gradbt = 0.0 
    if yt == 1:
        gradbt = 2*bt
    else:
        gradbt = 2*(bt - fnt)
    return gradbt

In [114]:
def alpha_grad(fpt,fnt,yt,alphat):
    '''
    Gradient with respect to alpha
    '''
    gradalphat = 0.0
    if yt == 1:
        gradalphat = -2*(alphat - fpt)
    else:
        gradalphat = -2*(alphat - fnt)
    return gradalphat

In [115]:
def loader(filename):
    '''
    Data file loader
    
    input:
        filename - filename
    
    output:
        x - sample features
        y - sample labels
    '''
    # raw data
    L = []
    with open(filename,'r') as file:
        for line in csv.reader(file, delimiter = ' '):
            line[0] = '0:'+line[0]
            line.remove('')
            L.append(dict(i.split(':') for i in line))
    df = pd.DataFrame(L,dtype=float).fillna(0)
    X = df.iloc[:,1:].values
    Y = df.iloc[:,0].values
    # centralize
    mean = np.mean(X,axis=1)
    #X = (X.transpose() - mean).transpose()
    # normalize
    norm = np.linalg.norm(X,axis=1)
    X = X/norm[:,None]
    # convert to binary class
    r = np.ptp(Y)
    index = np.argwhere(Y<r//2)
    INDEX = np.argwhere(Y>=r//2)
    Y[index] = -1
    Y[INDEX] = 1
    Y = Y.astype(int)
    return X,Y

In [116]:
def SOLAM(t,loss,batch,X,Y,L,lam,M,wt,at,bt,alphat):
    '''
    Stochastic Online AUC Maximization step
    
    input:
        T - total number of iteration
        F - objective function value
        loss - loss function
        pt - p at t
        wt - w at t
        at - a at t
        bt - b at t
        alphat - alpha at t
    output:
        W - record of each wt
        A - record of each at
        B - record of each bt
        ALPHA - record of each alphat
    '''
    # Loop in the batch
    peta = 1/t/M
    deta = np.sqrt(np.log(T)/T)
    for k in range(batch):
        
        # Update wt,at,bt
        prod = np.dot(wt,X[k])
        fpt = np.zeros(N+1)
        gfpt = np.zeros(N+1)
        fnt = np.zeros(N+1)
        gfnt = np.zeros(N+1)
        gradwt = 0.0
        gradat = 0.0
        gradbt = 0.0
        gradalphat = 0.0
        
        for i in range(N+1): # add up info of each i
            fpt[i],gfpt[i] = pos(i,prod,L) # partial info
            fnt[i],gfnt[i] = neg(loss,i,prod,L)
            gradwt += w_grad(gfpt[i],gfnt[i],Y[k],at[i],bt[i],alphat[i])
            gradat = a_grad(fpt[i],Y[k],at[i])
            gradbt = b_grad(fnt[i],Y[k],bt[i])
            gradalphat = alpha_grad(fpt[i],fnt[i],Y[k],alphat[i])
            at[i] -= deta*gradat/(N+1)/batch
            bt[i] -= deta*gradbt/(N+1)/batch
            alphat[i] += deta*gradalphat/(N+1)/batch
        
        wt = wt - peta*(gradwt*Y[k]*X[k]/(N+1)/batch + lam*wt) # step size as 1/t gradient descent
        
    wt = proj(wt,L/2)    
        
    return wt,at,bt,alphat

In [117]:
def prox(eta,loss,x,y,L,gamma,lam,wj,aj,bj,alphaj,bwt,bat,bbt,balphat):
    '''
    perform proximal guided gradient descent when receive an sample
    '''
    prod = np.inner(wj,x)
    fpt = np.zeros(N+1)
    gfpt = np.zeros(N+1)
    hfpt = np.zeros(N+1)
    fnt = np.zeros(N+1)
    gfnt = np.zeros(N+1)
    hfnt = np.zeros(N+1)
    gradwt = 0.0
    gradat = 0.0
    gradbt = 0.0
    gradalphat = 0.0
    hesswt = 0.0
    for i in range(N+1):
        fpt[i],gfpt[i],hfpt[i] = pos(i,prod,L)
        fnt[i],gfnt[i],hfnt[i] = neg(loss,i,prod,L)
        gradwt += w_grad(gfpt[i],gfnt[i],y,aj[i],bj[i],alphaj[i])# accumulate i
        hesswt += w_hess(hfpt[i],hfnt[i],y,aj[i],bj[i],alphaj[i])
        gradat = a_grad(fpt[i],y,aj[i])
        gradbt = b_grad(fnt[i],y,bj[i])
        gradalphat = alpha_grad(fpt[i],fnt[i],y,alphaj[i])
        aj[i] = aj[i] - eta*(gradat/(N+1)+gamma*(aj[i]-bat[i]))
        bj[i] = bj[i] - eta*(gradbt/(N+1)+gamma*(bj[i]-bbt[i]))
        alphaj[i] = alphaj[i] + eta*gradalphat/(N+1)
    hessian = hesswt*np.outer(x,x)
    eigen,_ = np.linalg.eig(hessian)
    
    print('minimum eigenvalue: %f' %(np.min(eigen)))
    wj = wj - eta*(gradwt*x*y/(N+1) + lam*wj + gamma*(wj - bwt))
    wj = proj(wj,L/2)
    #aJ = proj(aJ,1)
    #bJ = proj(bJ,1)
    #alphaJ = proj(alphaJ,1)
    
    return wj,aj,bj,alphaj

In [118]:
def PGSPD(t,loss,X,Y,L,gamma,lam,M,bwt,bat,bbt,balphat):
    '''
    Proximally Guided Stochastic Primal Dual Algorithm
    '''
    
    # initialize inner loop variables
    Wt = bwt+0.0
    At = bat+0.0
    Bt = bbt+0.0
    ALPHAt = balphat+0.0
    
    BWt = Wt+0.0
    BAt = At+0.0
    BBt = Bt+0.0
    BALPHAt = ALPHAt+0.0
    
    ETAt = 1/t/M # M is the bound for gradient
    # inner loop update at j
    for j in range(t): 
        # update inner loop variables
        Wt,At,Bt,ALPHAt = prox(ETAt,loss,X[j],Y[j],L,gamma,lam,Wt,At,Bt,ALPHAt,bwt,bat,bbt,balphat)
        BWt += Wt
        BAt += At
        BBt += Bt
        BALPHAt += ALPHAt
        
    # update outer loop variables
    bwt = BWt/t
    bat = BAt/t
    bbt = BBt/t
    balphat = BALPHAt/t
    
    return bwt,bat,bbt,balphat

In [119]:
def split(FEATURES,LABELS,folder,folders):
    
    if folder > folders:
        print('Exceed maximum folders!')
        return
    # load and split data
    #FEATURES,LABELS = loader(dataset)
    n,d = FEATURES.shape
    # regular portion of each folder
    portion = round(n/folders)
    start = portion*folder
    stop = portion*(folder+1)
    if np.abs(stop - n) < portion: # remainder occurs
        X_train = FEATURES[:start,:]
        Y_train = LABELS[:start]
        X_test = FEATURES[start:,:]
        Y_test = LABELS[start:]
    else:
        mask = np.ones(n, bool)
        mask[start:stop] = False
        X_train = FEATURES[mask,:]
        Y_train = LABELS[mask]
        X_test = FEATURES[start:stop]
        Y_test = LABELS[start:stop]
    # get dimensions of the data
    n,_ = X_train.shape
    # number of epoch
    epoch = T//n+1
    # augment by epoch
    #X_train_augmented = np.tile(X_train,(epoch,1)) # might have memory burden
    #Y_train_augmented = np.tile(Y_train,epoch)
    
    #return X_train_augmented,X_test,Y_train_augmented,Y_test
    return X_train,X_test,Y_train,Y_test

In [120]:
def demo(X_train_augmented,X_test,Y_train_augmented,Y_test,loss,alg,L = 2.0,gamma=0.0,lam=0.0,M=1.0,WT=0,AT=0,BT=0,ALPHAT=0):
    '''
    Run it to get results
    '''
    # define loss function
    if loss == 'hinge':
        loss = lambda x:max(0,1+L-2*L*x)
    elif loss == 'logistic':
        loss = lambda x:np.log(1+np.exp(L-2*L*x))
    else:
        print('Wrong loss function!')
        return
    
    # get dimensions of the data
    num,d = X_train_augmented.shape
    
    # initialize outer loop variables
    if type(WT) == int:
        WT = np.random.rand(d) # d is the dimension of the features
        AT = np.random.rand(N+1)
        BT = np.random.rand(N+1)
        ALPHAT = np.random.rand(N+1)

    # record auc
    roc_auc = np.zeros(T)
    # record time elapsed
    start_time = time.time()
    for t in range(1,T+1):
        
        if alg == 'PGSPD':
            if t<num:
                begin = (t*(t-1)//2)%num
                end = (t*(t+1)//2)%num
                if begin < end:
                    x_train = X_train_augmented[begin:end]
                    y_train = Y_train_augmented[begin:end]
                else: # need to think better
                    #s2 = time.time()
                    x_train = np.append(X_train_augmented[begin:],X_train_augmented[:end],axis=0)
                    y_train = np.append(Y_train_augmented[begin:],Y_train_augmented[:end],axis=0)
                    #e2 = time.time() - s2
                    #print('append time : %f' %(e1))
                x_train, y_train = shuffle(x_train,y_train)
                # update outer loop variables
                WT,AT,BT,ALPHAT = PGSPD(t,loss,x_train,y_train,L,gamma,lam,M,WT,AT,BT,ALPHAT)
            else:
                x_train, y_train = shuffle(X_train_augmented,Y_train_augmented)
                WT,AT,BT,ALPHAT = PGSPD(num,loss,x_train,y_train,L,gamma,lam,M,WT,AT,BT,ALPHAT)
            '''
            # sample a point
            begin = ((t-1)*num//batch)%num
            end = (t*num//batch)%num
            if begin < end:
                x_train = X_train_augmented[begin:end]
                y_train = Y_train_augmented[begin:end]
            else: # need to think better
                x_train = np.append(X_train_augmented[begin:],X_train_augmented[:end],axis=0)
                y_train = np.append(Y_train_augmented[begin:],Y_train_augmented[:end],axis=0)
            # update outer loop variables
            WT,AT,BT,ALPHAT = PGSPD(num//batch,loss,x_train,y_train,L,gamma,lam,M,WT,AT,BT,ALPHAT)
            '''
        elif alg == 'SOLAM':
            # sample a point
            begin = (t-1)*batch%num
            end = t*batch%num
            if begin < end:
                x_train = X_train_augmented[begin:end]
                y_train = Y_train_augmented[begin:end]
            else: # need to think better
                x_train = np.append(X_train_augmented[begin:],X_train_augmented[:end],axis=0)
                y_train = np.append(Y_train_augmented[begin:],Y_train_augmented[:end],axis=0)
            WT,AT,BT,ALPHAT = SOLAM(t,loss,batch,x_train,y_train,L,lam,M,WT,AT,BT,ALPHAT)
            
        fpr, tpr, _ = roc_curve(Y_test, np.dot(X_test,WT))
        roc_auc[t-1] = auc(fpr, tpr)
        if t%100 == 0:
            elapsed_time = time.time() - start_time
            print('iteration: %d AUC: %f time eplapsed: %f' %(t,roc_auc[t-1],elapsed_time))
            start_time = time.time()
    
    return WT,AT,BT,ALPHAT,roc_auc

In [121]:
def cv(dataset,loss,alg,folders,L = 2.0,gamma=0.0,lam=0.0,M=1.0,WT=0,AT=0,BT=0,ALPHAT=0):
    '''
    Cross validation
    '''
    # Load data set
    FEATURES,LABELS = loader(dataset) 
    
    # record auc
    AUC_ROC = np.zeros(folders)
    
    # cross validation
    for folder in range(folders):
        print('folder = %d' %(folder))
        X_train_augmented,X_test,Y_train_augmented,Y_test = split(FEATURES,LABELS,folder,folders)
        
        _,_,_,_,roc_auc = demo(X_train_augmented,X_test,Y_train_augmented,Y_test,loss,alg,L=L,gamma=gamma,lam=lam,M=M,WT=WT,AT=AT,BT=BT,ALPHAT=ALPHAT)
        AUC_ROC[folder] = np.max(roc_auc)
    print('auc score: %f +/- %f' %(np.mean(AUC_ROC),np.std(AUC_ROC)))
    return AUC_ROC

In [122]:
def gs(dataset,loss,alg,folders,L=[2.0],GAMMA=[0.0],LAM=[0.0],M=[1.0],WT=0,AT=0,BT=0,ALPHAT=0):
    '''
    Grid search! Wuss up fellas?!
    '''
    # Load data set
    FEATURES,LABELS = loader(dataset) 
    # record auc
    AUC_ROC = np.zeros((folders,len(L),len(GAMMA),len(LAM),len(M)))

    # cross validation
    for folder in range(folders):
        print('folder = %d' %(folder))
        X_train_augmented,X_test,Y_train_augmented,Y_test = split(FEATURES,LABELS,folder,folders)
        
        # bloody grid search!
        for l in range(len(L)):
            for gamma in range(len(GAMMA)):
                for lam in range(len(LAM)):
                    for m in range(len(M)):
                        print('current parameters: L = %.2f, GAMMA = %.2f, LAM = %.2f, M = %.2f' %(L[l],GAMMA[gamma],LAM[lam],M[m]))
                        _,_,_,_,roc_auc = demo(X_train_augmented,X_test,Y_train_augmented,Y_test,loss,alg,L=L[l],gamma=GAMMA[gamma],lam=LAM[lam],M=M[m],WT=WT,AT=AT,BT=BT,ALPHAT=ALPHAT)
                        AUC_ROC[folder,l,gamma,lam,m] = np.max(roc_auc)
    return AUC_ROC

In [123]:
N=10
T=200
batch=2

In [124]:
L=[0.02,0.2,2.0,20.0,200.0]
GAMMA = [0.01,0.1,1.0,10.0,100.0]
LAM = [0.01,0.1,1.0,10.0,100.0]
M = [0.01,0.1,1.0,10.0,100.0]

In [125]:
T = 100
diabetes_hinge = cv('diabetes','hinge','PGSPD',5,L = 1.0,gamma=0.1,lam=1.0,M=0.1)

folder = 0
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.086758
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.448475
minimum eigenvalue: -18.149207
minimum eigenvalue: -12.257249
minimum eigenvalue: -7.844667
minimum eigenvalue: -0.091208
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -18.811120
minimum eigenvalue: -34.844346
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -14.069957
minimum eigenvalue: -28.149164
minimum eigenvalue: -29.764250
minimum eigenvalue: -7.708109
minimum eigenvalue: -19.848458
minimum eigenvalue: -0.000000
minimum eigenvalue: -2.735890
minimum eigenvalue: -7.022293
minimum eigenvalue: -10.342616
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -12.436504
minimum eigenvalue: -10.040352
minimum eigenvalue: -0.000000
minimum eigenvalue

/Users/yangzhenhuan/PycharmProjects/AUC/venv/lib/python3.7/site-packages/ipykernel_launcher.py:31: ComplexWarning: Casting complex values to real discards the imaginary part


minimum eigenvalue: -2.927346
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -4.190714
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.400913
minimum eigenvalue: -13.152914
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.296148
minimum eigenvalue: -14.898912
minimum eigenvalue: -3.935484
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.661460
minimum eigenvalue: -11.641906
minimum eigenvalue: -10.698922
minimum eigenvalue: -7.861051
minimum eigenvalue: -1.428479
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.114878
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimu

minimum eigenvalue: -0.192387
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.106215
minimum eigenvalue: -0.509788
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.407411
minimum eigenvalue: -1.010976
minimum eigenvalue: -0.748002
minimum eigenvalue: -1.832649
minimum eigenvalue: -2.643162
minimum eigenvalue: -3.577016
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.120121
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.063739
minimum eigenvalue: -0.172084
minimum eigenvalue: -0.439829
minimum eigenvalue: -0.490855
minimum eigenvalue: -0.964982
minimum eigenvalue: -1.059219
minimum eigenvalue: -1.497604
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.062071
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.134384
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.110672
minimum eigenvalue: -0.464883
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum ei

minimum eigenvalue: -2.624071
minimum eigenvalue: -2.594391
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.412388
minimum eigenvalue: -0.617014
minimum eigenvalue: -0.578830
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.057216
minimum eigenvalue: -0.111165
minimum eigenvalue: -0.197733
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.035604
minimum eigenvalue: -0.090020
minimum eigenvalue: -0.157582
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.036800
minimum eigenvalue: -0.126341
minimum eigenvalue: -0.479918
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.207213
minimum eigenvalue: -0.504001
minimum eigenvalue: -0.698385
minimum eigenvalue: -2.158940
minimum eigenvalue: -0.000000
minimum ei

...

minimum eigenvalue: -0.000000
minimum eigenvalue: -0.013181
minimum eigenvalue: -0.004347
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.015922
minimum eigenvalue: -0.013961
minimum eigenvalue: -0.007304
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.006066
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.012818
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.012081
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.012355
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.013825
minimum eigenvalue: -0.020798
minimum eigenvalue: -0.027929
minimum eigenvalue: -0.035782
minimum eigenvalue: -0.053424
minimum eigenvalue: -0.081900
minimum eigenvalue: -0.114990
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.051022
minimum ei

minimum eigenvalue: -0.000000
minimum eigenvalue: -0.189867
minimum eigenvalue: -0.248867
minimum eigenvalue: -0.448349
minimum eigenvalue: -0.486767
minimum eigenvalue: -0.690758
minimum eigenvalue: -1.198548
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.659806
minimum eigenvalue: -0.876776
minimum eigenvalue: -1.054421
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.321383
minimum eigenvalue: -0.429800
minimum eigenvalue: -0.523296
minimum eigenvalue: -0.556674
minimum eigenvalue: -0.168897
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.118964
minimum eigenvalue: -0.187575
minimum eigenvalue: -0.244774
minimum eigenvalue: -0.269574
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.065655
minimum eigenvalue: -0.072258
minimum eigenvalue: -0.145916
minimum eigenvalue: -0.186161
minimum eigenvalue: -0.232145
minimum ei

minimum eigenvalue: -0.173912
minimum eigenvalue: -0.174872
minimum eigenvalue: -0.252665
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.136494
minimum eigenvalue: -0.157236
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.116227
minimum eigenvalue: -0.095618
minimum eigenvalue: -0.139041
minimum eigenvalue: -0.156892
minimum eigenvalue: -0.155988
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.084952
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.051172
minimum eigenvalue: -0.047790
minimum eigenvalue: -0.038603
minimum eigenvalue: -0.025002
minimum eigenvalue: -0.004018
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum ei

minimum eigenvalue: -0.048531
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.063227
minimum eigenvalue: -0.075335
minimum eigenvalue: -0.096719
minimum eigenvalue: -0.143974
minimum eigenvalue: -0.169264
minimum eigenvalue: -0.283275
minimum eigenvalue: -0.334646
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.201454
minimum eigenvalue: -0.210262
minimum eigenvalue: -0.278002
minimum eigenvalue: -0.360463
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.257032
minimum eigenvalue: -0.259106
minimum eigenvalue: -0.306171
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.210037
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.144319
minimum eigenvalue: -0.188269
minimum eigenvalue: -0.205981
minimum eigenvalue: -0.291721
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.110730
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.089562
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.076896
minimum eigenvalue: -0.000000
minimum ei

minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum eigenvalue: -0.000000
minimum ei